This notebook contains two sets of scraps accumulated through the project.

First there's a section of functions and analysis to compare activity level endpoint data and 

# Begin Data Build and Analysis (steps 2-4) for Activity Endpoint(s)

In [ ]:
# list of activities endpoints for reference

endpoints = ["activities/calories", "activities/caloriesBMR", "activities/steps", "activities/distance", 
             "activities/floors", "activities/elevation", "activities/minutesSedentary", 
             "activities/minutesLightlyActive", "activities/minutesFairlyActive", "activities/minutesVeryActive",
            "activities/activityCalories"]



def convertActiveMinuteData(data, endpoint):
    data_list = []
    
    endpoint = endpoint.replace(r"/", "-")
    column_name = endpoint.split("-")[1]
    for line in data[endpoint]:
        data_list.append({'end_date': datetime.datetime.strptime(line['dateTime'], "%Y-%m-%d"), column_name: int(line['value'])})
    df = pd.DataFrame(data_list)
    return df

import datetime

start_date = '2017-06-10' 
end_date = '2018-05-19'



endpoint = "activities/minutesVeryActive"

# first grab veryActiveMinutes
veryActiveData = getEndpointData(endpoint, start_date, end_date)

# grab date and minutes from veryActiveMinutes
veryActiveList = convertActiveMinuteData(veryActiveData, endpoint)

# then grab fairlyActiveMinutes
endpoint = "activities/minutesFairlyActive"
fairlyActiveData = getEndpointData(endpoint, start_date, end_date)

#grab date and minutes from fairlyActiveMinutes
fairlyActiveList = convertActiveMinuteData(fairlyActiveData, endpoint)


endpoint = "activities/minutesSedentary"
sedentaryActiveData = getEndpointData(endpoint, start_date, end_date)

#grab date and minutes from fairlyActiveMinutes
sedentaryActiveList = convertActiveMinuteData(sedentaryActiveData, endpoint)



endpoint = "activities/minutesLightlyActive"
lightlyActiveData = getEndpointData(endpoint, start_date, end_date)

#grab date and minutes from fairlyActiveMinutes
lightlyActiveList = convertActiveMinuteData(lightlyActiveData, endpoint)

# activity score. 
# dailyminutes (1440) = sedentaryMinutes + all active minutes

# of the waking time, how much was active, and how much was sedentary?

# use average number of asleep minutes - 407 to subtract from sedentaryMinutes and dailyMinutes

# what percentage of time was I active (out of awake minutes)?
# all_active_minutes / (1440-407)

# all_active_minutes / 1033

def calculatePercent(input_list, target):
    return target / sum(input_list) 




activity_data = pd.merge(veryActiveList, fairlyActiveList, on='end_date')
activity_data = pd.merge(activity_data, sedentaryActiveList, on='end_date')
activity_data = pd.merge(activity_data, lightlyActiveList, on='end_date')
activity_data = pd.merge(activity_data, sleep_df, on='end_date')

activity_data = activity_data.fillna(0)

print(activity_data)

# Investigate data further using Tableau

# Trying to see if very_active_minutes taken as a percent of total active time are 
# able to predict deep sleep


activity_data['very_active_percent'] = activity_data.apply(lambda x: calculatePercent([x['minutesLightlyActive'], x['minutesVeryActive'], x['minutesFairlyActive']], x['minutesVeryActive']), axis = 1)



plt.scatter(activity_data['very_active_percent'], activity_data['deep_minutes'])
plt.show()

It would seem that very_active_minutes are not a good predictor of the number of deep sleep minutes.

Do very active minutes decrease the number of waking events during the night?




plt.scatter(activity_data['very_active_percent'], activity_data['wake_count'])
plt.show()

plt.scatter((activity_data['minutesVeryActive']+activity_data['minutesFairlyActive']),activity_data['deep_minutes']/activity_data['minutes_asleep'])
plt.show()



endpoint = "activities/heart"
end_date = "2018-06-08"
start_date = "2018-05-09"

heart_rate_data = getEndpointData(endpoint, start_date, end_date)

pprint(heart_rate_data)



pprint(heart_rate_data['activities-heart'][0])

import datetime

heart_list = []

for record in heart_rate_data['activities-heart']:
    heart_dict = {}
    heart_dict['resting_heart_rate'] = record['value']['restingHeartRate']
    heart_dict['date_obj'] = datetime.datetime.strptime(record['dateTime'], "%Y-%m-%d").date()
    heart_dict['date_str'] = record['dateTime']
    for zone in record['value']['heartRateZones']:
        minute_name = zone['name'] + "_minutes"
        minutes = zone['minutes']
        calories_name = zone['name'] + "_calories"
        calories_out = zone['caloriesOut']
        heart_dict[minute_name] = minutes
        heart_dict[calories_name] = calories_out
    heart_list.append(heart_dict)
heart_df = pd.DataFrame(heart_list)

print(heart_df)

import matplotlib

plt.hist(heart_df['resting_heart_rate'], bins=7)
plt.show()

plt.hist(heart_df['Fat Burn_minutes'])
plt.show()

plt.hist(heart_df['Cardio_minutes'])
plt.show()

plt.scatter(heart_df['Cardio_minutes'], heart_df['resting_heart_rate'])
plt.show()


Below we have some code Melissa wrote to originally interact with the fitbit library (the one that runs into that 150 calls per hour limit.)  We're keeping the code here in case it's helpful as we're building out the other data sets.

In [ ]:
#yesterday2 = str((datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d"))
#today = str(datetime.now().strftime("%Y%m%d"))

#yesterday2 = ((datetime.now() - timedelta(days=1)))
#yesterday3 = (yesterday2 - timedelta(days=1))
#print(yesterday3)

current_day = "2018-05-18"


'''
These functions use the intra-day endpoint. 

CAUTION: Plan your calls wisely, or you will exceed 150 API calls per hour.

'''

#take a starting date and a total number of days as an input
# day needs to be in YYYY-MM-DD format
def pullFitBitData(start_date, days, call_type):
    #insert date error checking laterz
    print("Processing day: {}".format(start_date))
    current_date = start_date
    activity_df, heartRate_df = buildActivityData(current_date)
    day_counter = 0
    while day_counter < days:
        current_date = (datetime.datetime.strptime(current_date, "%Y-%m-%d") - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        print("Processing day: {}".format(current_date))
        activity_df2, heartRate_df2 = buildActivityData(current_date)
        activity_df = pd.concat([activity_df, activity_df2])
        heartRate_df = pd.concat([heartRate_df, heartRate_df2])
        day_counter += 1
    print("Ended processing on {}.".format(current_date))
    return activity_df, heartRate_df
        

with open('sleep_time_events_detail.txt', 'w') as outfile:
    json.dump(sleep_time_events_detail, outfile)

fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=current_day, detail_level='15min')
heartRateZones = (fit_statsHR['activities-heart'][0]['value']['heartRateZones'])

'''
column_names = heartRatePivot_df.columns.values
new_column_names = []
for name in column_names:
    new_name = name[1].replace(' ', '_')+'.'+name[0]
    new_column_names.append(new_name)
heartRatePivot_df.columns = new_column_names
print(heartRatePivot_df)
'''


def accumulateHeartData(current_day, heartRateZones):
    date_dict = {'date': [current_day, current_day, current_day, current_day]}
    date_df = pd.DataFrame(date_dict)
    
    heartRateZones_df = pd.DataFrame.from_records(heartRateZones)
    heartRateZones_df['date'] = date_df['date']
    heartRatePivot_df = heartRateZones_df.pivot(index='date', columns='name')
    column_names = heartRatePivot_df.columns.values
    new_column_names = []
    for name in column_names:
        new_name = name[1].replace(' ', '_')+'.'+name[0]
        new_column_names.append(new_name)
    heartRatePivot_df.columns = new_column_names
    return heartRatePivot_df

def buildActivityData(current_date):
    activity_stats = auth2_client.activities(date=current_date)
    pprint(activity_stats)
    activity_stats = activity_stats['summary']
    heartRateZones = activity_stats['heartRateZones']
    heartRate_df = accumulateHeartData(current_date, heartRateZones)
    del activity_stats['distances']
    del activity_stats['heartRateZones']
    activity_df = pd.DataFrame(activity_stats, index=[current_date])
    return activity_df, heartRate_df

activity_df, heartRate_df = buildActivityData("2018-05-18")

print(activity_df)

print(heartRate_df)

fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=current_day, detail_level='15min')
pprint(fit_statsHR)